In [1]:
from articles import yield_wiki_articles
from parse_wiki_markup import parse_wiki_markup

In [2]:
SMALL_FILE = r"C:\\Users\\Lenovo\\Downloads\\wikidump\\enwiki-20250501-pages-articles-multistream-index.txt.bz2"
BIG_FILE = r"C:\\Users\\Lenovo\\Downloads\\wikidump\\enwiki-20250501-pages-articles.xml.bz2"

articles = yield_wiki_articles(BIG_FILE)
    


In [3]:
article = next(articles)

In [4]:
parts = parse_wiki_markup(article['text'])

In [5]:
import spacy
import re
import sys
import subprocess
import requests
import json
from itertools import product

# --- SETUP ---
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    subprocess.check_call([sys.executable, "-m", "spacy", "download", "en_core_web_sm"])
    nlp = spacy.load("en_core_web_sm")

# --- OLLAMA BRIDGE ---
class OllamaVerifier:
    def __init__(self, model="llama3"):
        self.api_url = "http://localhost:11434/api/generate"
        self.model = model

    def verify_is_person(self, name, context_snippet):
        """
        Returns True (1) if LLM thinks it's a person, False (0) otherwise.
        """
        prompt = (
            f"Analyze the text snippet below.\n"
            f"Text: \"{context_snippet}\"\n"
            f"Question: Is the entity '{name}' strictly a human being in this specific context? "
            f"Ignore statues, tombs, wars, or organizations.\n"
            f"Reply with exactly one digit: 1 for Yes, 0 for No."
        )

        payload = {
            "model": self.model,
            "prompt": prompt,
            "stream": False,
            "options": {"temperature": 0} # Deterministic
        }

        try:
            response = requests.post(self.api_url, json=payload, timeout=2)
            result = response.json().get('response', '').strip()
            # Parse for 1 or 0
            if '1' in result: return True
            return False
        except Exception as e:
            # If Ollama fails/times out, default to keeping it (fail open) 
            # or discarding it (fail closed). Here we fail closed to be safe.
            print(f"⚠️ Ollama Error: {e}")
            return False

# --- EXTRACTOR ---
class BaseWikiExtractor:
    
    RE_YEAR = re.compile(r'\b(1\d{3}|20[0-2]\d)\b')

    MODEL="llama3:latest"
    
    # Heuristics for "High Probability" vs "Low Probability"
    # If a name contains these, it's "Low Prob" and needs LLM verification
    RISKY_KEYWORDS = {
        "king", "queen", "prince", "princess", "pope", "saint", "st.", "baron", "lord", # Titles
        "the", "of", "de", "van", "von", "bin", "al", # Particles (often messy)
        "wrath", "treaty", "war", "battle", "tomb", "statue", "memorial", # Objects/Events
        "v", "i", "ii", "iii", "iv", "x", "vi", # Roman numerals (e.g. Nicholas V)
        "great", "terrible", "bold", "lion" # Epithets
    }

    def __init__(self, model=MODEL):
        self.ollama = OllamaVerifier(model=model) # Change model as needed

    def _clean_text(self, text):
        if not isinstance(text, str): return ""
        text = re.sub(r'\{\{.*?\}\}', '', text)
        text = re.sub(r'\[\[(File|Image|Category):.*?\]\]', '', text, flags=re.IGNORECASE)
        text = re.sub(r'\[\[(?:[^|\]]*\|)?([^\]]+)\]\]', r'\1', text)
        text = re.sub(r'==+.*?==+', '', text)
        text = re.sub(r'\w+="[^"]+"', '', text) 
        text = re.sub(r"'{2,}", "", text) 
        return " ".join(text.split())

    def _is_suspicious_name(self, text):
        """
        Dual Channel Logic:
        Returns TRUE if the name is 'Risky' (Channel 2).
        Returns FALSE if the name is 'High Probability' (Channel 1).
        """
        lower = text.lower()
        tokens = set(re.split(r'\W+', lower))
        
        # 1. Check for Risky Keywords (Titles, Particles, Epithets)
        if not tokens.isdisjoint(self.RISKY_KEYWORDS):
            return True
            
        # 2. Check for Roman Numerals at the end (e.g. "Nicholas V")
        if re.search(r'\b[IVX]+\b$', text):
            return True
            
        # 3. Check for Non-Title Case (e.g. "túmulo de...")
        # A valid person name in English Wiki is almost always Title Cased.
        if not text[0].isupper():
            return True
            
        return False

    def _find_triplets_in_text(self, text):
        if len(text) > 50000: text = text[:50000]
        
        doc = nlp(text)
        triplets = []
        
        for sent in doc.sents:
            persons = []
            locations = []
            years = []

            # 1. NER Extraction
            for ent in sent.ents:
                if ent.label_ == "PERSON":
                    # Basic cleanup
                    clean_name = ent.text.strip()
                    if len(clean_name) > 3 and len(clean_name.split()) >= 2:
                        persons.append((clean_name, ent.start, ent.end))
                
                elif ent.label_ in ["GPE", "LOC"]:
                    if len(ent.text) > 2 and not re.search(r'\d', ent.text):
                        locations.append((ent.text, ent.start))
                
                elif ent.label_ == "DATE":
                    match = self.RE_YEAR.search(ent.text)
                    if match:
                        years.append((match.group(1), ent.start))

            # 2. Logic & Verification
            if persons and locations and years:
                for p_text, p_start, p_end in persons:
                    
                    # --- CHANNEL SELECTION ---
                    is_valid_person = True
                    
                    if self._is_suspicious_name(p_text):
                        # CHANNEL 2: LOW PROBABILITY -> LLM VERIFICATION
                        # Extract context snippet (the full sentence)
                        context = sent.text
                        # Call Ollama (returns 0 or 1)
                        # print(f"🔍 Verifying suspicious entity: '{p_text}'...")
                        if not self.ollama.verify_is_person(p_text, context):
                            is_valid_person = False
                    
                    # CHANNEL 1: High Prob -> Accepted automatically
                    
                    if is_valid_person:
                        # Proximity Logic
                        best_loc = min(locations, key=lambda x: abs(x[1] - p_start))
                        best_year = min(years, key=lambda x: abs(x[1] - p_start))
                        
                        if abs(best_loc[1] - p_start) < 25 and abs(best_year[1] - p_start) < 25:
                            triplets.append((p_text, best_loc[0], best_year[0]))

        return triplets

In [6]:
unique_results = set()
found = 0
while True:
    try:
        article = next(articles)
        parts = parse_wiki_markup(article['text'])
        results = extract_all_triplets(parts)
        if results:
            print(results)
            break
            found += len(results)
            #print(f"\nFound {len(results)} triplets:")
            for r in results:
                try:
                    unique_results.add(r)
                    found += 1
                except Exception as ee:
                    continue
        print(f"Total triplets found: {found}", end='\r')
    except Exception as ae:
        continue

KeyboardInterrupt: 

In [7]:
list(unique_results)[0:50]

[]

In [ ]:
TARGET='Thopmson'
MIN_YEAR = 1650
MAX_YEAR = 1720
for a in list(unique_results): 
    try:
        year = int(a[2])
        if 1650 > year < 2025:
            if TARGET in str(a):
                print(a)

ValueError: invalid literal for int() with base 10: 'Category:20th-century'

In [ ]:
from gliner import GLiNER

# Load a small, fast model (runs on CPU easily)
model = GLiNER.from_pretrained("urchade/gliner_small-v2.1")

text = "While visiting the Metro in 2024, Philip Alston met with officials."
labels = ["person", "location", "year"] # You define these on the fly!

entities = model.predict_entities(text, labels)

for entity in entities:
    print(f"{entity['text']} => {entity['label']}")
    # Output:
    # Philip Alston => person
    # Metro => location
    # 2024 => year

ModuleNotFoundError: No module named 'gliner'

In [ ]:
import instructor
from openai import OpenAI
from pydantic import BaseModel
from typing import List

# Define the EXACT structure you want
class BioEvent(BaseModel):
    person: str
    location: str
    year: int

class ExtractionResult(BaseModel):
    events: List[BioEvent]

# Patch the client (works with Ollama too!)
client = instructor.patch(OpenAI(base_url="http://localhost:11434/v1", api_key="ollama"))

def extract_with_llm(text):
    return client.chat.completions.create(
        model="llama3:latest"
    ,
        response_model=ExtractionResult,
        messages=[
            {"role": "user", "content": f"Extract biographical events from: {text}"}
        ]
    )

# Result is a guaranteed valid python object, no regex needed.
# result.events[0].person == "Philip Alston"

ModuleNotFoundError: No module named 'instructor'